# Fichier: LECTURE ET ANALYSE DES DATASETS
### Importation des librairies

In [ ]:
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

from plotly import __version__
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
from plotly.offline.offline import _plot_html
import plotly.graph_objs as go
print (__version__) # requires version >= 1.9.0
init_notebook_mode(connected=True)

from mpl_toolkits import mplot3d
import matplotlib.pylab as plt
from matplotlib.pylab import rcParams
%matplotlib notebook
#%matplotlib inline

from utils import series_supp as ss
from utils import data_factory as df
from utils import plot

In [ ]:
cwd = os.getcwd()
os.chdir(cwd)
print(cwd)
factory = df.DataFactory(cwd)
ploter = plot.Plot(None)

RG24 = ss.SeriesSupp(cwd, factory, "RG24")
RG1 = ss.SeriesSupp(cwd, factory, "RG1")
GW = ss.SeriesSupp(cwd, factory, "GW")

### Importation des datasets depuis les fichiers de stockage csv

In [ ]:
GW.import_dataset()
RG24.import_dataset()

#### Parametres du plot
Année et mois

In [ ]:
ploter.change_mode(3)
#years, months = [2015], [3]

GW.reset_dataset()
RG24.reset_dataset()

#GW.years = years
#GW.months = months
GW.reset_months()
GW.reset_years()

#RG24.years = years
#RG24.months = months
RG24.reset_months()
RG24.reset_years()

GW.split_data_years()
GW.split_data_months()
#GW.split_data_weeks()

RG24.split_data_years()
#RG24.split_data_months()
#RG24.split_data_weeks()

#RG24.dict_round()
#GW.dict_round()

#GW.dict_norm()
#RG24.dict_norm()

#### Plot RG

In [ ]:
#ploter.plot_scatter(RG24.get_data())

#### Plot GW

In [ ]:
ploter.change_mode(2)

In [ ]:
GW.get_data()

In [ ]:
ploter.plot_scatter_by_capteur(GW.get_data(), ["GW_267"])